# **Introduction to Text Summarization**:
**There are two major types of text summarization**:
  -	Extractive: summaries consist of text from the original document.   Extractive summaries are relatively easy to achieve. The whole concept is to find the relevant parts of the input text. 
  
  -	Abstractive: summaries can contain novel sequences of text not necessarily taken from the original text. Abstractive summaries are harder to achieve, even generating a very simple text is quite a challenge.


* This notebook is dedicated for covering different techniques of Extractive summarization.

In [7]:
import pandas as pd
from IPython.core.display import HTML
import numpy as np
import textwrap
import nltk
import re
import string
import heapq
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
#from google.colab import data_table

import networkx as nx
from nltk.cluster.util import cosine_distance

#data_table.enable_dataframe_formatter()

# import pandas as pd
# import io
# import numpy as np
# import textwrap
# import nltk
# from nltk.corpus import stopwords
# from nltk import word_tokenize
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [8]:
# from google.colab import drive
# drive.mount('drive')

In [9]:
df = pd.read_json("shuffled_hashed_rbc_dataset.json")
df = df.transpose()
df.head()
# df = df[['article_overview','article_text']]
# df.head()

,overview,headline,body,category,tags
78d8b9491b72c4e0d0add8f77fa5cc90,РНПК стала ключевым перестраховщиком российски...,Reuters назвал нового главного перестраховщика...,Reuters назвал нового главного перестраховщик...,economics,"[страхование, страховщики, суда, нефть, эмбарг..."
16449d4a36b1db5e87183cb407a54552,Минстрой разработал план по увеличению использ...,Власти предложили строить дома и школы по техн...,Власти предложили строить дома и школы по тех...,business,"[Строительство, Минстрой, сталь, металлургия, ..."
695434577d5c1ff776910676cef4b6f7,"По его мнению, необходимость наличия такого ор...",Рогозин заявил о необходимости противоспутнико...,Рогозин заявил о необходимости противоспутник...,tech,"[Роскосмос, Дмитрий Рогозин, спутник]"
703096d803b367ec3bebc96f271273a1,"Речь идет о перестроении исторического блока, ...",Число уроков истории в школах не увеличат из-з...,Число уроков истории в школах не увеличат из-...,politics,"[Минпросвещения, история, школы]"
21295f8f3febaebc9ea5e3c4077f9de8,Суд присяжных присудил Деппу $10 млн в качеств...,Суд признал Хёрд виновной в клевете и обязал в...,Суд признал Хёрд виновной в клевете и обязал ...,society,"[Джонни Депп, суд, приговор, клевета]"


4. After that, we will download the needed files from NLTK.

In [10]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Data Exploration**:

In [11]:
df.head(3)

,overview,headline,body,category,tags
78d8b9491b72c4e0d0add8f77fa5cc90,РНПК стала ключевым перестраховщиком российски...,Reuters назвал нового главного перестраховщика...,Reuters назвал нового главного перестраховщик...,economics,"[страхование, страховщики, суда, нефть, эмбарг..."
16449d4a36b1db5e87183cb407a54552,Минстрой разработал план по увеличению использ...,Власти предложили строить дома и школы по техн...,Власти предложили строить дома и школы по тех...,business,"[Строительство, Минстрой, сталь, металлургия, ..."
695434577d5c1ff776910676cef4b6f7,"По его мнению, необходимость наличия такого ор...",Рогозин заявил о необходимости противоспутнико...,Рогозин заявил о необходимости противоспутник...,tech,"[Роскосмос, Дмитрий Рогозин, спутник]"


In addition, It has been found that we have five entries with no articles. This might be due to some interraption in the internet connection while performing the webscraping.

## **Supporting Functions**

This section consists of some supporting functions that will be used frequently in this notebook.

In [20]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace = False, fix_sentence_endings= True)

In [21]:
def hilight_summary(article,summary_sentences,sentences_format = 0):

    original_article_overview = article['article_overview'].iloc[0]
    original_sentences = article['article_text'].iloc[0]

    if(sentences_format == 0):
        original_article_overview = " ".join([sentence for sentence in nltk.sent_tokenize(original_article_overview)])
        original_sentences = [sentence for sentence in nltk.sent_tokenize(original_sentences)]
        summary_sentences = [sentence for sentence in nltk.sent_tokenize(summary_sentences)]

    text = ''
    display(HTML(original_article_overview.replace(original_article_overview, f'<h4>{original_article_overview}</h4>')))
    display(HTML(f'<h2>Summary</h2>'))
    for sentence in original_sentences:
        if sentence in summary_sentences:
            text += ' ' + sentence.replace(sentence, f"<mark>{sentence}</mark>")
        else:
            text += ' ' + sentence

    display(HTML(f""" {text} """))

In [22]:
def hilight_summary(original_sentences,summary_sentences,sentences_format = 0):

    # original_article_overview = article['article_overview'].iloc[0]
    # original_sentences = article['article_text'].iloc[0]

    if(sentences_format == 0):
        # original_article_overview = " ".join([sentence for sentence in nltk.sent_tokenize(original_article_overview)])
        original_sentences = [sentence for sentence in nltk.sent_tokenize(original_sentences)]
        summary_sentences = [sentence for sentence in nltk.sent_tokenize(summary_sentences)]

    text = ''
    # display(HTML(original_article_overview.replace(original_article_overview, f'<h4>{original_article_overview}</h4>')))
    display(HTML(f'<h2>Summary</h2>'))
    for sentence in original_sentences:
        if sentence in summary_sentences:
            text += ' ' + sentence.replace(sentence, f"<mark>{sentence}</mark>")
        else:
            text += ' ' + sentence

    display(HTML(f""" {text} """))

The below code utilizes a method that was developed in the __supporting functions__ to highlight the extracted summary out of the original text.

In [26]:
df.describe()

,overview,headline,body,category,tags
count,1534,1534,1534,1534,1534
unique,1520,1518,1534,6,1494
top,Несмотря на «резкую» просьбу Шольца присоедини...,В 15 регионах сообщили о погибших в ходе военн...,Reuters назвал нового главного перестраховщик...,finance,"[Банки, Ипотека]"
freq,2,2,1,333,5


To smoothen the process, it has been decided to remove these NA values.

In addition, it has been decided to perform some data cleanup before starting any summarization process, particularly cleaning up the new lines.

# **Summarization Methods**:
In the following sections, couple of library-based summarization techniques (standard tradetional) will be presented along with their summarization performance. These summarization methodologies are:


1.   TF-IDF for Text Summarization
2.   Luhn algorithm
3.   Cosine similarity
4.   SUMY ( A.   TextRankSummarizer , B.   LSA Summarizer )
5.   Pysummarization
6.   BERT


Other summarization techniques, that are more advanced, will be presented in a separate notebook.

## **1. TF-IDF for Text Summarization**

With this approach, we will performing the follwing:
1. Split the text into sentences.
2. Score each sentence.
3. Rank each sentence by scores.
4. Summary = top scoring sentences.


For demonstration purposes, we wil start with explaining the approach step by step. After that, we will build a function to apply the summarization on all the articles.

#### **A. Perform Summarization on One Data Sample:**

To begin with, let us use one article for summarization

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

for headline in df.headline:
    title_of_doc = headline
    doc = df[df.headline == headline]['article_text']

Now, we will tokenize the text into sentences

In [28]:
sents = nltk.sent_tokenize(doc.iloc[0])

After that, we will create TF-IDF vectorizer object.

Note that we are using russian stopwords and L1 normlization. This is to insure that we will not be bias toward longer sentences with higher score "simply due to the fact that they contain more words".

In [16]:
featurizer = TfidfVectorizer(stop_words= stopwords.words('russian'), norm= 'l1')


Next, we call fit transform which give us back our TF-IDF matrix.

In [17]:
X = featurizer.fit_transform(sents)

The following code will show the summary along with the model score. As can be seen, it takes a single row from the TF-IDF matrix. Only non-zero values are considered once you have those values a mean function is applied.

In [18]:
def get_sentence_score(tfidf_row):
    # return the average of the non-zero values
    # of the tf-idf vector representation of a sentence
    x = tfidf_row[tfidf_row != 0]
    return x.mean()

In [24]:
#compute scores for each sentence
scores = np.zeros(len(sents))
for i in range(len(sents)):
    score = get_sentence_score(X[i,:])
    scores[i] = score

#sort the scores
sort_idx = np.argsort(-scores)

#print summary
for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (scores[i], sents[i])))

0.25: Позднее «Донбасс» сменил курс.
0.17: В Кремле сочли инцидент в Азовском море провокацией.
0.17: НАТО Россия вооруженный конфликт провокации Минобороны
0.12: Он также обвинил Россию в стягивании войск к границам Украины.
0.11: Корабль был взят на сопровождение российским кораблем береговой
охраны.


#### **B. Building a function to Perform Summarization on all data samples::**

- We prepared a function to summarize using tf_idf

- The first step is to tokenize the text into sentences

- Next we call fit transform which give us back our TF-IDF matrix (X)

- Finally preparing the summarization that include some extra steps such as scoring and sorting. then printing the highest scored 5 sentences

In [25]:
'''
creating the TF-IDF vectorizer object again in case we deicde to go for 
this approach directly in stead of the step by step approach. (note that we are
using russian stopwords and l1 normlization this insure that we will not be 
bias toward longer sentences which in this case would have a higher score. 
simply due to the fact that they contain more words.)
'''
featurizer = TfidfVectorizer(stop_words= stopwords.words('russian'), norm= 'l1')


#definning the scoring function again as we need it inside the summarize_TF_IDF funciton
#The function is intended to score a sentence given it's TF-IDF representation. 
def get_sentence_score(tfidf_row):
    # return the average of the non-zero values
    # of the tf-idf vector representation of a sentence
    x = tfidf_row[tfidf_row != 0]
    return x.mean()



def summarize_TF_IDF(text):
    # extract sentences
    sents = nltk.sent_tokenize(text)
    
    #perform tf-idf
    X = featurizer.fit_transform(sents)
    
    #compute scores for each sentence
    scores = np.zeros(len(sents))
    for i in range(len(sents)):
        score = get_sentence_score(X[i,:])
        scores[i] = score
    
    #sort the scores
    sort_idx = np.argsort(-scores)
    
    #return summary
    summary_text = str('')
    for i in sort_idx[:5]:
        # summary_text = summary_text + ("%.2f: %s " % (scores[i], sents[i]))
        summary_text = summary_text + ("%s " % (sents[i]))
    
    summary_text = re.sub(r'\n', ' ', summary_text)
    return(summary_text)

Now, we will apply the summarization to all the documents.

In [ ]:
tf_idf_summaries = []
for article in df['article_text']:
    tf_idf_summaries.append(summarize_TF_IDF(article))

df['TF_IDF_Summary'] = tf_idf_summaries

In [30]:
for headline in df.headline:
    title_of_doc = headline
    doc = df[df.headline == headline]

print(wrap(doc['headline'].iloc[0]))
print("========= summary ==========")
print(wrap(doc['TF_IDF_Summary'].iloc[0]))

Минобороны заявило о риске вооруженного конфликта с НАТО из-за
провокаций
========= summary ==========
Позднее «Донбасс» сменил курс. В Кремле сочли инцидент в Азовском море
провокацией. НАТО Россия вооруженный конфликт провокации Минобороны Он
также обвинил Россию в стягивании войск к границам Украины. Корабль
был взят на сопровождение российским кораблем береговой охраны.


In [31]:
df[['headline', 'article_text', 'TF_IDF_Summary']]

,headline,body,TF_IDF_Summary
78d8b9491b72c4e0d0add8f77fa5cc90,Reuters назвал нового главного перестраховщика...,Reuters назвал нового главного перестраховщик...,Также в Лондоне находится Lloyd's of London — ...
16449d4a36b1db5e87183cb407a54552,Власти предложили строить дома и школы по техн...,Власти предложили строить дома и школы по тех...,"м жилья. РБК направил запрос в МЧС. м, ежегодн..."
695434577d5c1ff776910676cef4b6f7,Рогозин заявил о необходимости противоспутнико...,Рогозин заявил о необходимости противоспутник...,Роскосмос Дмитрий Рогозин спутник «Роскосмос» ...
703096d803b367ec3bebc96f271273a1,Число уроков истории в школах не увеличат из-з...,Число уроков истории в школах не увеличат из-...,"«Сегодня основные этапы, основные события нове..."
21295f8f3febaebc9ea5e3c4077f9de8,Суд признал Хёрд виновной в клевете и обязал в...,Суд признал Хёрд виновной в клевете и обязал ...,"Тот, в свою очередь, назвал эти заявления клев..."
...,...,...,...
ec312179c358e03f9c8ac193a8c95367,Маск предложил разблокировать Трампа в Twitter,Маск предложил разблокировать Трампа в Twitte...,После этого Twitter станет частной компанией. ...
a69ecfe9c2a889291bf00c7d8b3b78a5,"Новак сообщил, что половина из 54 покупателей ...","Новак сообщил, что половина из 54 покупателей...",«Это механизм во времени растянутый. Новое тре...
4bdd54e239712b87b8f5c1eb694ca109,Азербайджан и Казахстан допустили сотрудничест...,Азербайджан и Казахстан допустили сотрудничес...,В 2018 году был запущен третий азербайджанский...
dc7e8f603555f6de0ff564944d1a6dfe,Попавший под санкции Нисанов вышел из совета д...,Попавший под санкции Нисанов вышел из совета ...,"64% акций спорткомплекса, принадлежавших столи..."


## **2. Luhn Algorithm**

This is one of the earliest approaches of text summarization. Luhn proposed that the significance of each word in a document signifies how important it is. The idea is that any sentence with maximum occurances of the highest frequency words(Stopwords) and least occurances are not important to the meaning of document than others. Although it is not considered very accurate approach.

The Detailes of the algorithim can be found in the source link below.

source:
https://iq.opengenus.org/luhns-heuristic-method-for-text-summarization/


Resources:
- https://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf
- https://www.cs.cmu.edu/~nasmith/LS2/das-martins.07.pdf


#### **A. Perform Summarization on One Data Sample:**

In [32]:
original_text = doc['article_text'].iloc[0]
original_text

' Минобороны заявило о риске вооруженного конфликта с НАТО из-за провокаций Замглавы Минобороны Фомин допустил вооруженный конфликт из-за провокаций НАТО у границ В 2021 году присутствие в черноморской зоне кораблей НАТО «фактически приобрело постоянный характер», а интенсивность действий разведывательной авиации альянса выросла более чем на 60%, сообщили в Минобороны России Провокации НАТО у российских границ могут перерасти в вооруженный конфликт. Об этом заявил заместитель министра обороны России Александр Фомин на брифинге, передает «РИА Новости». «В последнее время альянс перешел к практике прямых провокаций, сопряженных с высоким риском перерастания в вооруженное противостояние», — сказал он. В качестве примера он привел инцидент в Черном море 23 июня, когда британский эсминец УРО «Дефендер» попытался проникнуть в территориальные воды России в районе мыса Фиолент в Крыму. Он указал на то, что действия британского корабля обеспечивал стратегический разведывательный самолет RC-135 

Unlike some algorithims, this algorithim needs stop words treatment

In [33]:
stopwords = nltk.corpus.stopwords.words('russian')
print(stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Now, we will define a function that does the required preprocessing for Luhn algorithim

In [34]:
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
  formatted_text = ' '.join(element for element in tokens)

  return formatted_text

In [35]:
formatted_text = preprocess(original_text)
formatted_text

'минобороны заявило риске вооруженного конфликта нато из-за провокаций замглавы минобороны фомин допустил вооруженный конфликт из-за провокаций нато границ 2021 году присутствие черноморской зоне кораблей нато « фактически приобрело постоянный характер » интенсивность действий разведывательной авиации альянса выросла 60 сообщили минобороны россии провокации нато российских границ могут перерасти вооруженный конфликт заявил заместитель министра обороны россии александр фомин брифинге передает « риа новости » « последнее время альянс перешел практике прямых провокаций сопряженных высоким риском перерастания вооруженное противостояние » — сказал качестве примера привел инцидент черном море 23 июня британский эсминец уро « дефендер » попытался проникнуть территориальные воды россии районе мыса фиолент крыму указал действия британского корабля обеспечивал стратегический разведывательный самолет rc-135 сша целом 2021 году интенсивность задействования разведывательной авиации районе черного м

Below function will be used to calculate sentences score

In [36]:
def calculate_sentences_score(sentences, important_words, distance):
  scores = []
  sentence_index = 0

  for sentence in [nltk.word_tokenize(sentence) for sentence in sentences]:
    #print('------------')
    #print(sentence)

    word_index = []
    for word in important_words:
      #print(word)
      try:
        word_index.append(sentence.index(word))
      except ValueError:
        pass

    word_index.sort()
    #print(word_index)

    if len(word_index) == 0:
      continue

    # [0, 1, 5]
    groups_list = []
    group = [word_index[0]]
    i = 1 # 3
    while i < len(word_index): # 3
      # first execution: 1 - 0 = 1
      # second execution: 2 - 1 = 1
      if word_index[i] - word_index[i - 1] < distance:
        group.append(word_index[i])
        #print('group', group)
      else:
        groups_list.append(group[:])
        group = [word_index[i]]
        #print('group', group)
      i += 1
    groups_list.append(group)
    #print('all groups', groups_list)

    max_group_score = 0
    for g in groups_list:
      #print(g)
      important_words_in_group = len(g)
      total_words_in_group = g[-1] - g[0] + 1
      score = 1.0 * important_words_in_group**2 / total_words_in_group
      #print('group score', score)

      if score > max_group_score:
        max_group_score = score

    scores.append((max_group_score, sentence_index))
    sentence_index += 1

  #print('final scores', scores)
  return scores

In addition, we have developped a function that summarizes the text based on the scores

In [37]:
def summarize_Luhn(text, top_n_words, distance, number_of_sentences, percentage = 0):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
    #print(original_sentences)
    formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
    #print(formatted_sentences)
    words = [word for sentence in formatted_sentences for word in nltk.word_tokenize(sentence)]
    #print(words)
    frequency = nltk.FreqDist(words)
    #print(frequency)
    #return frequency
    top_n_words = [word[0] for word in frequency.most_common(top_n_words)]
    #print(top_n_words)
    sentences_score = calculate_sentences_score(formatted_sentences, top_n_words, distance)
    #print(sentences_score)
    if percentage > 0:
    best_sentences = heapq.nlargest(int(len(formatted_sentences) * percentage), sentences_score)
    else:  
    best_sentences = heapq.nlargest(number_of_sentences, sentences_score)
    #print(best_sentences)
    best_sentences = [original_sentences[i] for (score, i) in best_sentences]
    #print(best_sentences)
    return original_sentences, best_sentences, sentences_score

After defining the needed functions, it is the time for applying the summerization. 

In [38]:
original_sentences, best_sentences, sentences_score = summarize_Luhn(original_text, 5, 2, 4)

In [39]:
original_doc_sentences = original_sentences
original_sentences

[' Минобороны заявило о риске вооруженного конфликта с НАТО из-за провокаций Замглавы Минобороны Фомин допустил вооруженный конфликт из-за провокаций НАТО у границ В 2021 году присутствие в черноморской зоне кораблей НАТО «фактически приобрело постоянный характер», а интенсивность действий разведывательной авиации альянса выросла более чем на 60%, сообщили в Минобороны России Провокации НАТО у российских границ могут перерасти в вооруженный конфликт.',
 'Об этом заявил заместитель министра обороны России Александр Фомин на брифинге, передает «РИА Новости».',
 '«В последнее время альянс перешел к практике прямых провокаций, сопряженных с высоким риском перерастания в вооруженное противостояние», — сказал он.',
 'В качестве примера он привел инцидент в Черном море 23 июня, когда британский эсминец УРО «Дефендер» попытался проникнуть в территориальные воды России в районе мыса Фиолент в Крыму.',
 'Он указал на то, что действия британского корабля обеспечивал стратегический разведывательны

In [40]:
best_sentences

['В Кремле сочли инцидент в Азовском море провокацией.',
 'Корабль был взят на сопровождение российским кораблем береговой охраны.',
 'Министр обороны Украины Алексей Резников выразил недоумение, что ФСБ увидела угрозу в поисково-спасательном судне без оружия.',
 'Киев расценил инцидент как информационную атаку со стороны России.']

In [41]:
sentences_score

[(1.0, 0),
 (1.0, 1),
 (1.0, 2),
 (1.0, 3),
 (1.0, 4),
 (1.0, 5),
 (1.0, 6),
 (1.0, 7),
 (1.0, 8),
 (1.0, 9),
 (1.0, 10),
 (2.0, 11),
 (1.0, 12),
 (2.0, 13),
 (1.0, 14),
 (1.0, 15),
 (1.0, 16)]

In [42]:
hilight_summary(original_sentences,best_sentences,sentences_format=1)

Now, we will apply the summarization to all the articles and store the summary in seperate column

In [43]:
Luhn_Summaries = []
for article in df['article_text']:
    original_sentences, best_sentences, sentences_score = summarize_Luhn(article, 5, 2, 4)
    summary_text = str('')
    for i in best_sentences:
        summary_text = summary_text + ("%s " % (i))
    Luhn_Summaries.append(summary_text)

df['Luhn_Summary'] = Luhn_Summaries

Drawing a randome article to see the summary

In [44]:
sample_article2 = df.sample()
hilight_summary(sample_article2['article_text'].iloc[0],sample_article2['Luhn_Summary'].iloc[0])

## **3. Cosine similarity**
Cosine similarity is one of the metric to measure the text-similarity between two documents irrespective of their size in Natural language Processing. A word is represented into a vector form. The text documents are represented in n-dimensional vector space.

Mathematically, Cosine similarity metric measures the cosine of the angle between two n-dimensional vectors projected in a multi-dimensional space. The Cosine similarity of two documents will range from 0 to 1. If the Cosine similarity score is 1, it means two vectors have the same orientation. The value closer to 0 indicates that the two documents have less similarity.

Resources:
- https://en.wikipedia.org/wiki/Cosine_similarity 
- https://studymachinelearning.com/cosine-similarity-text-similarity-metric/


In [45]:
original_sentences = original_doc_sentences

We will continue using the same stopwords and preprocessing function we define earlier for Lhun. Also, we will keep using the same original text used previously.

Now, we will define a function to calculate sentence similarity using cosine similarity.

In [46]:
def calculate_sentence_similarity(sentence1, sentence2):
    words1 = [word for word in nltk.word_tokenize(sentence1)]
    words2 = [word for word in nltk.word_tokenize(sentence2)]
    #print(words1)
    #print(words2)

    all_words = list(set(words1 + words2))
    #print(all_words)

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    #print(vector1)
    #print(vector2)

    for word in words1: # Bag of words
    #print(word)
    vector1[all_words.index(word)] += 1
    for word in words2:
    vector2[all_words.index(word)] += 1

    #print(vector1)
    #print(vector2)

    return 1 - cosine_distance(vector1, vector2)

We are using the same original sentences and showing the similarity score for first and fourth setences

In [47]:
calculate_sentence_similarity(original_sentences[0], original_sentences[3])

0.28223355773559566

Below is a function that creates the similarity matrix:
- The higher the value, the greater the similarity between the sentences
- The more words in common, the greater the similarity

In [48]:
def calculate_similarity_matrix(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    #print(similarity_matrix)
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue
            similarity_matrix[i][j] = calculate_sentence_similarity(sentences[i], sentences[j])
    return similarity_matrix

In [49]:
calculate_similarity_matrix(original_sentences)

array([[0.        , 0.18579547, 0.26547739, 0.28223356, 0.09853293,
        0.41376009, 0.10931247, 0.39225297, 0.41862002, 0.11377602,
        0.15676493, 0.0623177 , 0.11172584, 0.16422155, 0.15422843,
        0.0623177 , 0.1393466 , 0.14079065, 0.21501648, 0.28444006,
        0.16728422, 0.30845685, 0.23188676, 0.19788526, 0.11883518,
        0.21285545, 0.32180719, 0.44248489],
       [0.18579547, 0.        , 0.22680461, 0.17766726, 0.1767767 ,
        0.17213259, 0.06537205, 0.13900961, 0.15811388, 0.13608276,
        0.25      , 0.1490712 , 0.26726124, 0.07856742, 0.21081851,
        0.1490712 , 0.16666667, 0.04811252, 0.25717225, 0.36288737,
        0.28583098, 0.10540926, 0.18490007, 0.17213259, 0.07106691,
        0.21821789, 0.19245009, 0.        ],
       [0.26547739, 0.22680461, 0.        , 0.31914237, 0.14433757,
        0.24595493, 0.16012815, 0.31212739, 0.21516574, 0.05555556,
        0.30618622, 0.06085806, 0.21821789, 0.19245009, 0.25819889,
        0.06085806, 0.1814

Since we sow what is the similarity matrix, we can use the same function to evaluate and get the best summary.

In [57]:
def summarize_cosine(text, number_of_sentences, percentage = 0):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
    formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
    similarity_matrix = calculate_similarity_matrix(formatted_sentences)

    similarity_graph = nx.from_numpy_array(similarity_matrix)


    scores = nx.pagerank_numpy(similarity_graph)
    ordered_scores = sorted(((scores[i], score) for i, score in enumerate(original_sentences)), reverse=True)

    if percentage > 0:
        number_of_sentences = int(len(formatted_sentences) * percentage)
        number_of_sentences = min(number_of_sentences, len(ordered_scores))
    best_sentences = []

    for sentence in range(number_of_sentences):
        best_sentences.append(ordered_scores[sentence][1])
  
    return original_sentences, best_sentences, ordered_scores

After developing all needed functions for evaluation and summarization, we need to perform the summary on the selected article. We selected the top four highly ranked sentences to for the summary.

In [58]:
original_sentences, best_sentences_cosine, scores = summarize_cosine(original_text, 4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  # This is added back by InteractiveShellApp.init_path()


Now, let us take a look into the highly ranked sentences.

In [59]:
best_sentences_cosine

[' Минобороны заявило о риске вооруженного конфликта с НАТО из-за провокаций Замглавы Минобороны Фомин допустил вооруженный конфликт из-за провокаций НАТО у границ В 2021 году присутствие в черноморской зоне кораблей НАТО «фактически приобрело постоянный характер», а интенсивность действий разведывательной авиации альянса выросла более чем на 60%, сообщили в Минобороны России Провокации НАТО у российских границ могут перерасти в вооруженный конфликт.',
 'Президент России также говорил, что США пришли с ракетами к «порогу нашего дома».',
 'Министр иностранных дел России Сергей Лавров заявил, что НАТО «замахнулась на порог нашего дома», с чем Россия не будет мириться.',
 'Путин выразил «определенную озабоченность» в связи с крупномасштабными учениями НАТО в Черном море на границе с Россией.']

Below are all the sentences along with their scours

It is the time to visualize the summary highlighted on the original context

In [61]:
hilight_summary(original_sentences,best_sentences_cosine,sentences_format=1)

## **4. Sumy**

__Sumy__ is a library to extract summaries from HTML pages or plain texts. By utilizing __Sumy__, we will be performing two summarization techniques:
* TextRankSummarizer
* LSA Summarizer


For more information about Sumy, you could visit the official library's documentation page.
- library's pypi page: https://pypi.org/project/sumy/



### **A. Sumy Installation** 

In [79]:
!pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 10.1 MB 26.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21712 sha256=e4b3e0d06e543618a6c82050c1d5efcb41869011911797a5df7d175da27542e0
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=880ae0c325c47c748aea5d53dc0e2b2286efd1f051cadd3d24bc57a35fe064fd
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built breadability pycountry


In [80]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

### **B. TextRankSummarizer**
Text Rank is a kind of graph-based ranking algorithm used for recommendation purposes. TextRank is used in various applications where text sentences are involved. It worked on the ranking of text sentences and recursively computed based on information available in the entire text.

TextRank builds the graph related to the text. In a graph, each sentence is considered as vertex and each vertex is linked to the other vertex. These vertices cast a vote for another vertex. The importance of each vertex is defined by the higher number of votes. This importance is Here goal is to rank the sentences and this rank.

<br/>

Resources:
- https://medium.com/@ondenyi.eric/extractive-text-summarization-techniques-with-sumy-3d3b127a0a32
- https://machinelearninggeek.com/text-summarization-using-python/

##### **B.1 Perform Summarization on One Data Sample:**
Apply textranksummarizer to our test paragraph

In [81]:
def text_rank_summarizer(text, number_of_sentences):
    summarizer = TextRankSummarizer()
    parser = PlaintextParser.from_string(
      text,
      Tokenizer("russian"))
    summary = summarizer(parser.document, sentences_count= number_of_sentences)

    summary_text = ''
    for s in summary:
        summary_text = summary_text +' '+ str(s)
  
    return summary_text.strip()

In [82]:
hilight_summary(doc['article_text'].iloc[0],text_rank_summarizer(doc['article_text'].iloc[0],5))

Apply the textranksummarizer to all article

In [83]:
text_rank_summaries = []
for article in df['article_text']:
    summary_text = text_rank_summarizer(article,5)
    text_rank_summaries.append(summary_text)

df['Sumy_Text_Rank_Summarizer_Summary'] = text_rank_summaries

In [84]:
sumdem = df.sample()
# sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0]
hilight_summary(sumdem['article_text'].iloc[0],sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0])

### **C. LSA Summarizer**
Latent Semantic Analysis is based on Singular value decomposition(SVD). It reduces the data into lower-dimensional space. It performs spatial decomposition and captures information in a singular vector and the magnitude of so singular vector will represent the importance.


##### **C.1 Perform Summarization on One Data Sample:**
Apply textranksummarizer to our test paragraph

In [85]:
def LSA_summarizer(text, number_of_sentences):
    summarizer = LsaSummarizer()
    parser = PlaintextParser.from_string(
      text,
      Tokenizer("russian"))
    summary = summarizer(parser.document, sentences_count= number_of_sentences)

    summary_text = ''
    for s in summary:
        summary_text = summary_text + ' '+ str(s)

    return summary_text.strip()

In [86]:
hilight_summary(doc['article_text'].iloc[0],LSA_summarizer(doc['article_text'].iloc[0],5))

##### **B.2 Building a function to Perform Summarization on all data samples:**

Apply the LSA Summarizer to all article

In [87]:
lsa_summaries = []
for article in df['article_text']:
    summary_text = LSA_summarizer(article,5)
    lsa_summaries.append(summary_text)

df['LSA_Summarizer_Summary'] = lsa_summaries

In [88]:
sumdem = df.sample()
# sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0]
hilight_summary(sumdem['article_text'].iloc[0],sumdem['LSA_Summarizer_Summary'].iloc[0])

## **5. Pysummarization**

__pysummarization__ is Python library for the automatic summarization, document abstraction, and text filtering.

- https://pypi.org/project/pysummarization/

### **A. Pysummarization Installation**


In [89]:
!pip install pysummarization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 147 kB/s 
  Created wheel for pysummarization: filename=pysummarization-1.1.8-py3-none-any.whl size=59453 sha256=d2f9f84f3be8f5deb29d7b637ff5f98f849e145bae273d0c06d871601964b885
  Stored in directory: /root/.cache/pip/wheels/da/14/3e/02d15001af23ca877c5149b66280a605e5cdbbe76972598afa
Successfully built pysummarization


In [90]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor

### **B. Perform Summarization on One Data Sample:**
Apply textranksummarizer to our test paragraph

In [91]:
auto_abstractor = AutoAbstractor()
auto_abstractor.tokenizable_doc = SimpleTokenizer()
auto_abstractor.delimiter_list = [".", "\n"]
abstractable_doc = TopNRankAbstractor()

In [92]:
summary = auto_abstractor.summarize(doc['article_text'].iloc[0], abstractable_doc)

In [93]:
summary

{'scoring_data': [(0, 38.75438596491228),
  (1, 14.0),
  (2, 20.0),
  (3, 23.14814814814815),
  (5, 8.909090909090908),
  (10, 4.454545454545454),
  (11, 3.2666666666666666),
  (24, 3.125),
  (29, 3.6),
  (30, 3.5714285714285716)],
 'summarize_result': [' Минобороны заявило о риске вооруженного конфликта с НАТО из-за провокаций Замглавы Минобороны Фомин допустил вооруженный конфликт из-за провокаций НАТО у границ В 2021 году присутствие в черноморской зоне кораблей НАТО «фактически приобрело постоянный характер», а интенсивность действий разведывательной авиации альянса выросла более чем на 60%, сообщили в Минобороны России Провокации НАТО у российских границ могут перерасти в вооруженный конфликт.\n',
  ' Об этом заявил заместитель министра обороны России Александр Фомин на брифинге, передает «РИА Новости».\n',
  ' «В последнее время альянс перешел к практике прямых провокаций, сопряженных с высоким риском перерастания в вооруженное противостояние», — сказал он.\n',
  ' В качестве при

applying it to our document 

In [94]:
best_sentences = ''
for sentence in summary['summarize_result']:
    best_sentences = best_sentences  + sentence

In [95]:
hilight_summary(doc['article_text'].iloc[0],best_sentences)

### **C. Building a function to Perform Summarization on all data samples:**

In [96]:
pysummarization_list = []
for article in df['article_text']:
    summary = auto_abstractor.summarize(article, abstractable_doc)
    best_sentences = ''
    for sentence in summary['summarize_result']:
        best_sentences = best_sentences  + sentence

    pysummarization_list.append(best_sentences.strip())

df['pysummarization_summary'] = pysummarization_list

In [97]:
sumdem = df.sample()
# sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0]
hilight_summary(sumdem['article_text'].iloc[0],sumdem['pysummarization_summary'].iloc[0])

In [ ]:
sample_article3 = df.sample()
print("TFIDF")
hilight_summary(sample_article3['article_text'].iloc[0],sample_article3['TF_IDF_Summary'].iloc[0])
print("=========================")
print("Luhn")
hilight_summary(sample_article3['article_text'].iloc[0],sample_article3['Luhn_Summary'].iloc[0])
print("pysummarization_summary")
hilight_summary(sample_article3['article_text'].iloc[0],sample_article3['pysummarization_summary'].iloc[0])
print("=========================")


## **6- Bidirectional Encoder Representations from Transformers (BERT)**
As defined in the Wikipedia, __BERT__ is a transformer-based machine learning technique for natural language processing (NLP) pre-training developed by Google.
Although, __BERT__ is not considered as part of the classical language models for text summarization. However, we will include a lighter implementation of it as a transition between this notebook and the advanced ones.

- https://pypi.org/project/bert-extractive-summarizer/

### **A. BERT Installation**

In [100]:
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.3 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [101]:
!pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-w15gierb
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-w15gierb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.20.0.dev0-py3-none-any.whl size=4394469 sha256=88783c76ee0ef5923a25a72e5f69de4e71a9a2d732b1d86e2b673e14aae88805
  Stored in directory: /tmp/pip-ephem-wheel-cache-ou2o_gsj/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.19.4
    Uninstalling transformers-4.19.4:
      Successfully uninstalled transformers-4.19.4


In [102]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 5.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=56d2ab615a667507a6edfc03fc004014f163fb2c1b7d1155668841ba83be94fd
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


### **B. Summarization on One Data Sample:**

In [103]:
from transformers import BigBirdTokenizer
from summarizer import Summarizer

In [104]:
summarizer = Summarizer()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [105]:
summary = summarizer(doc['article_text'].iloc[0])

In [106]:
hilight_summary(doc['article_text'].iloc[0],summary)

In [108]:
df.to_pickle('extractive_summarization_one.pkl')